
# <div style="text-align: right"> Random Forest from scratch. </div>

---

<div style="text-align: right"> Geoff Counihan - Oct 6, 2017 </div>

### Notes

---

Inhereted my decision tree class to create a random forest classifier. There are a few modifications.

    1. Sample with replacement
    2. Create, predict, and average multiple tree predictions
    3. Modify split function to spit off n_feat random features
    
__Additions__: Entropy, Test Cases

In [111]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import operator

In [182]:
Xy = pd.read_csv('./sonar.all-data.csv',header=None)
Xy[60] = Xy[60].map({'R':0,'M':1})
X = np.array(Xy.iloc[:,:-1])
y = np.array(Xy.iloc[:,-1])
Xy = np.array(Xy)

In [5]:
# iris = load_iris()
# X = iris.data[:,:2]
# y = iris.target

# Xy = np.column_stack((X,y))
# Xy_point = Xy[:3]
# print(Xy_point)

[[ 5.1  3.5  0. ]
 [ 4.9  3.   0. ]
 [ 4.7  3.2  0. ]]


### Data manipulations

Random Forests are powerful because they ensemble a large number of weak learners. For each tree to be different from another two manipulations are performed

---

__Sample with replacement__ - create a ficticous dataset sampled with replacement from the original data

In [138]:
def samp(Xy,ratio):
    n = int(np.round(len(Xy) * ratio))
    idx = np.random.randint(Xy.shape[0],size=n)
    return Xy[idx,:]

In [141]:
samp(Xy,1).shape

(208, 61)

In [341]:
class random_forest(decision_tree):
    def __init__(self, num_trees, max_depth=2, min_num_split=30, sample_ratio=1):
        self.max_depth = max_depth
        self.min_num_sample = min_num_split
        self.num_trees = num_trees
        self.ratio = sample_ratio
        
    def build_tree(self, Xy):
        '''Recursively build tree, unclear if this is the correct way
        
        '''
        self.root = self.best_split(Xy)
        #print(self.root)
        self.split_branch(self.root, 1) # i don't understand how this is working, pointed to node?
        #print(self.root)
        return self.root
    
    def best_split(self, Xy):
        classes = np.unique(Xy[:,-1])
        best_feat = 999
        best_val = 999
        best_score = 999
        best_groups = None
        n_feats = np.random.choice(Xy.shape[1]-1, self.n_feat, replace=False)
        #print(n_feats)
        for feat in n_feats:
            for i in Xy:
                groups = self.split(feat, i[feat], Xy)
                #print(groups)
                gini = self.gini_score(groups, classes)
                #print('feat {}, valued < {}, scored {}'.format(feat,i[feat], gini))
                if gini < best_score:
                    best_feat = feat
                    best_val = i[feat]
                    best_score = gini
                    best_groups = groups
        output = {}
        output['feat'] = best_feat
        output['val'] = best_val
        output['groups'] = best_groups
        return output
    
    def samp(self, Xy, ratio=1):
        n = int(np.round(len(Xy) * ratio))
        idx = np.random.randint(Xy.shape[0],size=n)
        return Xy[idx,:]
        
    def fit(self, X, y):
        '''Save training data.
        
        '''
        self.X = X
        self.y = y
        self.Xy = np.column_stack((X, y))

        self.n_feat = int(np.sqrt(X.shape[1]))
        
        self.trees = [self.build_tree(self.samp(self.Xy)) for i in range(self.num_trees)]
        
    def predict(self, X_test):
        self.y_preds = np.array([]).reshape(0,X_test.shape[0])
        for root in self.trees:
            y_pred = np.array([])
            for i in X_test:
                y_pred = np.append(y_pred,self.predict_sample(root,i))
            #print(y_pred.shape)
            self.y_preds = np.vstack((self.y_preds,y_pred))
        self.avg_preds = np.rint(self.y_preds.mean(axis=0))
        return self.avg_preds
        
        

In [279]:
# dt = decision_tree(max_depth=2,min_num_split=30)
# dt.fit(X,y)

In [306]:
np.rint(rf_pred)

array([ 1.,  0.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
        1.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,
        1.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,
        1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  1.])

In [280]:
X_test.shape[0]

52

In [281]:
rf = random_forest(num_trees=3)

In [282]:
rf.fit(X,y)

[41 20 16 19 12 30 48]
[ 2 26 38 23 33 13 57]
[22  8 58 52 17 10 11]
[42 32 27  1 17 55 24]
[38 54 49 48 41 14  7]
[57 18 32 13 16 41 37]
[36 41 59 42 13 28 45]
[24 57  7 59 29 49 30]


In [283]:
rf.predict(X_test)

(52,)
(52,)
(52,)


array([ 0.66666667,  0.        ,  0.66666667,  1.        ,  0.33333333,
        0.        ,  0.33333333,  0.66666667,  1.        ,  0.66666667,
        1.        ,  0.66666667,  0.66666667,  0.66666667,  0.        ,
        1.        ,  1.        ,  0.        ,  0.        ,  0.66666667,
        1.        ,  1.        ,  0.66666667,  0.66666667,  0.33333333,
        0.        ,  0.33333333,  0.33333333,  0.66666667,  0.66666667,
        1.        ,  1.        ,  0.66666667,  0.33333333,  1.        ,
        0.66666667,  1.        ,  1.        ,  0.        ,  0.66666667,
        1.        ,  0.66666667,  1.        ,  0.66666667,  0.        ,
        0.33333333,  0.66666667,  0.33333333,  0.66666667,  0.        ,
        0.        ,  0.        ])

In [249]:
rf.trees

[{'feat': 10,
  'left': {'feat': 51, 'left': 0.0, 'right': 1.0, 'val': 0.017600000000000001},
  'right': {'feat': 35, 'left': 1.0, 'right': 0.0, 'val': 0.4577},
  'val': 0.17419999999999999},
 {'feat': 46,
  'left': {'feat': 52,
   'left': 1.0,
   'right': 0.0,
   'val': 0.00050000000000000001},
  'right': {'feat': 12, 'left': 0.0, 'right': 1.0, 'val': 0.2087},
  'val': 0.0608},
 {'feat': 12,
  'left': {'feat': 51,
   'left': 0.0,
   'right': 0.0,
   'val': 0.0071999999999999998},
  'right': {'feat': 8, 'left': 0.0, 'right': 1.0, 'val': 0.074999999999999997},
  'val': 0.247}]

In [258]:
X_test.shape[1]

60

In [259]:
Xy.shape[1]

61

In [256]:
X.shape

(208, 60)

### Create class.

---

__Tie__ - Added to modify behavior when there is a tie for majority class

### Test.

---

In [252]:
from sklearn.model_selection import train_test_split

Xy = pd.read_csv('./sonar.all-data.csv',header=None)
Xy[60] = Xy[60].map({'R':0,'M':1})
X = np.array(Xy.iloc[:,:-1])
y = np.array(Xy.iloc[:,-1])
Xy = np.array(Xy)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=35)

In [321]:
dt = decision_tree(max_depth=3,min_num_split=30)
dt.fit(X_train,y_train)
dt_pred = dt.predict(X_test)

In [342]:
rf = random_forest(max_depth=2,num_trees=100,min_num_split=30)
rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)

### Display tree.
---

In [943]:
dt.display_tree()

[feat1 < 5.40]
	[feat2 < 2.70]
		[1.0]
		[0.0]
	[feat1 < 6.20]
		[1.0]
		[2.0]


### Compare performance

---

In [285]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [292]:
clf = DecisionTreeClassifier(max_depth=2,min_samples_split=30)
clf.fit(X_train,y_train)
sk_pred = clf.predict(X_test)

### Sample points

---

In [295]:
a = X_test[:3]

In [343]:
print('sk_pred: {}'.format(clf.predict(a)))
print('dt_pred: {}'.format(dt.predict(a)))
print('rf_pred: {}'.format(rf.predict(a)))
print('true: {}'.format(y_test[:3]))

sk_pred: [1 0 0]
dt_pred: [ 1.  0.  0.]
rf_pred: [ 1.  0.  1.]
true: [0 0 0]


### Accuracy differences

---

I'm unclear how sklearn differs. Will need to look deeper

In [344]:
def accuracy(pred,true):
    correct = 0
    pred_len = len(pred)
    for i in range(pred_len):
        if pred[i] == true[i]:
            correct += 1
    return correct/pred_len

In [345]:
accuracy(dt_pred,y_test)

0.6538461538461539

In [346]:
accuracy(rf_pred,y_test)

0.6538461538461539

In [347]:
accuracy(sk_pred,y_test)

0.6153846153846154

In [348]:
list(zip(rf_pred,dt_pred,sk_pred,y_test))

[(1.0, 1.0, 1, 0),
 (0.0, 0.0, 0, 0),
 (1.0, 0.0, 0, 0),
 (1.0, 1.0, 1, 1),
 (1.0, 1.0, 1, 1),
 (0.0, 0.0, 0, 0),
 (1.0, 1.0, 1, 0),
 (1.0, 0.0, 1, 0),
 (1.0, 1.0, 1, 0),
 (0.0, 0.0, 0, 0),
 (1.0, 0.0, 0, 1),
 (1.0, 1.0, 1, 0),
 (1.0, 1.0, 1, 1),
 (1.0, 1.0, 1, 1),
 (0.0, 0.0, 0, 0),
 (1.0, 1.0, 1, 1),
 (1.0, 1.0, 1, 1),
 (0.0, 0.0, 0, 0),
 (0.0, 0.0, 0, 1),
 (1.0, 1.0, 0, 1),
 (1.0, 1.0, 1, 1),
 (1.0, 1.0, 1, 1),
 (1.0, 1.0, 1, 0),
 (1.0, 1.0, 1, 1),
 (0.0, 0.0, 0, 0),
 (0.0, 0.0, 0, 0),
 (1.0, 0.0, 0, 0),
 (1.0, 0.0, 0, 1),
 (1.0, 0.0, 0, 1),
 (0.0, 0.0, 0, 1),
 (1.0, 0.0, 0, 0),
 (1.0, 1.0, 1, 1),
 (1.0, 1.0, 1, 1),
 (1.0, 1.0, 1, 0),
 (1.0, 1.0, 1, 0),
 (0.0, 0.0, 0, 0),
 (1.0, 0.0, 0, 1),
 (1.0, 1.0, 1, 1),
 (0.0, 0.0, 0, 0),
 (1.0, 1.0, 1, 0),
 (1.0, 1.0, 1, 0),
 (1.0, 1.0, 1, 0),
 (1.0, 1.0, 1, 1),
 (0.0, 1.0, 1, 1),
 (0.0, 0.0, 0, 0),
 (0.0, 0.0, 0, 0),
 (0.0, 1.0, 1, 0),
 (1.0, 1.0, 1, 0),
 (1.0, 1.0, 1, 1),
 (0.0, 0.0, 0, 0),
 (0.0, 0.0, 0, 0),
 (0.0, 0.0, 0, 0)]

In [333]:
class decision_tree(object):
    def __init__(self, max_depth, min_num_split):
        self.max_depth = max_depth
        self.min_num_sample = min_num_split
    
    def gini(self, X):
        if np.min(X) < 0:
            X -= np.amin(X)
        #X = X.flatten()
        X += .00001
        X = np.sort(X)
        i = np.arange(1,X.shape[0]+1)
        n = X.shape[0]
        return ((np.sum((2 * i - n - 1) * X)) / (n * np.sum(X)))
    
    def gini_score(self, groups, classes):
        n_samples = sum([len(group) for group in groups])
        gini = 0
        for group in groups:
            size = float(len(group))
            if size == 0:
                continue
            score = 0.0
            #print(size)
            for class_val in classes:
                #print(group.shape)
                p = (group[:,-1] == class_val).sum() / size
                #print(p)
                score += p * p
            gini += (1.0 - score) * (size / n_samples)
            #print(gini)
        return gini
    
    def split(self, feat, val, Xy):
#         Xi_left = np.array([]).reshape(0,3)
#         Xi_right = np.array([]).reshape(0,3)
        Xi_left = np.array([]).reshape(0,self.Xy.shape[1])
        Xi_right = np.array([]).reshape(0,self.Xy.shape[1])
        for i in Xy:
            #print(i.shape)
            if i[feat] <= val:
                Xi_left = np.vstack((Xi_left,i))
            if i[feat] > val:
                Xi_right = np.vstack((Xi_right,i))
        return Xi_left, Xi_right
    
    def best_split(self, Xy):
        classes = np.unique(Xy[:,-1])
        best_feat = 999
        best_val = 999
        best_score = 999
        best_groups = None
        for feat in range(Xy.shape[1]-1):
            for i in Xy:
                groups = self.split(feat, i[feat], Xy)
                #print(groups)
                gini = self.gini_score(groups, classes)
                #print('feat {}, valued < {}, scored {}'.format(feat,i[feat], gini))
                if gini < best_score:
                    best_feat = feat
                    best_val = i[feat]
                    best_score = gini
                    best_groups = groups
        output = {}
        output['feat'] = best_feat
        output['val'] = best_val
        output['groups'] = best_groups
        return output
    
    def terminal_node(self, group):
        # errored out: couldn't np.unique(nothing) or something - doesn't happen all the time
        #print(group[:,-1])
        classes, counts = np.unique(group[:,-1],return_counts=True)
        return classes[np.argmax(counts)]
            
    def split_branch(self, node, depth):
        left_node, right_node = node['groups']
        del(node['groups'])
        if not isinstance(left_node,np.ndarray) or not isinstance(right_node,np.ndarray):
            node['left'] = node['right'] = self.terminal_node(left_node + right_node)
            return
        if depth >= self.max_depth:
            node['left'] = self.terminal_node(left_node)
            node['right'] = self.terminal_node(right_node)
            return
        if len(left_node) <= self.min_num_sample:
            node['left'] = self.terminal_node(left_node)
        else:
            node['left'] = self.best_split(left_node)
            self.split_branch(node['left'], depth+1)
        if len(right_node) <= self.min_num_sample:
            node['right'] = self.terminal_node(right_node)
        else:
            node['right'] = self.best_split(right_node)
            self.split_branch(node['right'], depth+1)
    
    def build_tree(self):
        '''Recursively build tree, unclear if this is the correct way
        
        '''
        
        self.root = self.best_split(self.Xy)
        #print(self.root)
        self.split_branch(self.root, 1) # i don't understand how this is working, pointed to node?
        #print(self.root)
        return self.root
    
    def fit(self,X,y):
        '''Save training data.
        
        '''
        self.X = X
        self.y = y
        self.Xy = np.column_stack((X, y))
        self.build_tree()

    def display_tree(self, depth=0):
        if isinstance(self.root,dict):
            print('{}[feat{} < {:.2f}]'.format(depth*'\t',(self.root['feat']+1), self.root['val']))
            display_tree(self.root['left'], depth+1)
            display_tree(self.root['right'], depth+1)
        else:
            print('{}[{}]'.format(depth*'\t', self.root))
            
    def predict_sample(self, node, sample):
        #print(node)
        if sample[node['feat']] < node['val']:
            if isinstance(node['left'],dict):
                return self.predict_sample(node['left'],sample)
            else:
                return node['left']
        else:
            if isinstance(node['right'],dict):
                return self.predict_sample(node['right'],sample)
            else:
                return node['right']
    
    def predict(self, X_test):
        self.y_pred = np.array([])
        for i in X_test:
            #print(i)
            self.y_pred = np.append(self.y_pred,self.predict_sample(self.root,i))
        return self.y_pred
        